In [ ]:
import os
import tensorflow as tf
import cv2
import imghdr # DeprecationWarning: 
from PIL import Image
from matplotlib import pyplot as plt
from typing import List, Tuple, Dict, Any, Union

In [ ]:
DATA_DIR = 'data'
IMAGE_EXTENSIONS = ["JPEG", "PNG", "GIF", "BMP"]
LOG_DIR = 'logs'

In [ ]:
def sanitize_images(files: List[str]) -> None:
    
    deleted_files = 0  # counter for deleted files
    
    for filename in os.listdir(files):
        file_path = os.path.join(files, filename)

        try:
            with Image.open(file_path) as img:
                if img.format not in IMAGE_EXTENSIONS:
                    print(f'Deleting (not a valid format) --> {filename}')
                    os.remove(file_path)
                    deleted_files += 1
                    continue
                print(filename, img.format, "%dx%d" % img.size, img.mode)
        except IOError:
            print(f'Deleting --> {filename}')
            os.remove(file_path)
            deleted_files += 1  # increment the counter
            continue
        
        
        try:
            x = cv2.imread(file_path)
            if x is None:
                print(f'Deleting: cannot be read by CV2 - {filename}')
                os.remove(file_path)
                continue
        except Exception as e:
            print(f'Deleting: cannot be read by CV2 - {filename}')
            os.remove(file_path)
            continue
        

        try:
            # Get file size in KB
            file_size = os.path.getsize(file_path) / 1024  # in KB
            if file_size < 10:
                print(f'Deleting (less than 10KB) --> {filename}')
                os.remove(file_path)
                deleted_files += 1  # increment the counter
                continue
        except Exception as e:
            print(f'Deleting: cannot get file size - {filename}')
            os.remove(file_path)
            continue
            
    print(f'Total deleted files: {deleted_files}')


In [ ]:
# Loop through the `data` directory and clean up data
for directory in os.listdir(DATA_DIR):
    print(f'--> Cleaning up {directory} directory <--')
    directory: List[str] = os.path.join(DATA_DIR, directory)
    sanitize_images(directory)
    print(f'--> Done cleaning up {directory} directory <--')


In [ ]:
# Avoid OOM error by setting GPU Memory Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')
data_iter = data.as_numpy_iterator()
batch = data_iter.next()

In [ ]:
# Sample batch images
fig, ax = plt.subplots(ncols=4,figsize=(20, 20))
for i, img in enumerate(batch[0][:4]):
    ax[i].imshow(img.astype(np.uint8))
    ax[i].title.set_text(f'label: {batch[1][i]}')

In [ ]:
scaled_data = data.map(lambda x, y: (x / 255.0, y))
# data = data.map(lambda x, y: (tf.image.resize(x, (224, 224)), y))
# data = data.map(lambda x, y: (tf.keras.applications.mobilenet_v2.preprocess_input(x), y))
# data = data.map(lambda x, y: (tf.keras.applications.resnet50.preprocess_input(x), y))
scaled_iterator = scaled_data.as_numpy_iterator()
scaled_batch = scaled_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4,figsize=(20, 20))
for i, img in enumerate(scaled_batch[0][:4]):
    ax[i].imshow(img)
    ax[i].title.set_text(f'label: {scaled_batch[1][i]}')


In [ ]:
len(scaled_batch)

In [ ]:
train_size = int(len(scaled_batch)*0.7)+1
val_size = int(len(scaled_batch)*0.2 )+1
test_size= int( len(scaled_batch)*0.1)+1
print(train_size, val_size, test_size)

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
len(test)

### 3. Deep Model

3.1 Build Deep Learning Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(2))

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss=tf.losses.BinaryFocalCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

3.3 Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='validation_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc='upper right')
plt.show()

In [ ]:
fig = plt.figure( )
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='validation_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc='upper right')
plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    x, y = batch
    y_pred = model.predict(x)
    pre.update_state(y, y_pred)
    re.update_state(y, y_pred)
    acc.update_state(y, y_pred)
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}')

In [ ]:
# img = cv2.imread('data/happy/who-is-happier.jpg')
# img = cv2.imread('data/sad/secret-signs-of-loneliness-01-1440x810.jpg')
# plt.imshow(img)
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
# plt.show()

In [ ]:
# resized_img = cv2.resize(img, (256, 256))
# plt.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
# plt.show()

In [ ]:
# np.expand_dims(resized_img, 0).shape

In [ ]:
# prediction = model.predict(np.expand_dims(resized_img/255, 0))

In [ ]:
# print(prediction)
# if prediction > 0.5:
#     print('Happy')
# else:
#     print('Sad')